In [10]:
from IPython.display import HTML, display, clear_output
import ipywidgets as widgets
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
import io

display(HTML("""
<style>
  body {
    background: linear-gradient(135deg, #007BFF, #E6F0FF);
    font-family: 'Arial', sans-serif;
  }
  .custom-button {
    background: #007BFF !important;
    border-radius: 12px !important;
    color: white !important;
    font-weight: bold !important;
    font-size: 16px !important;
    padding: 12px 25px !important;
    margin: 10px !important;
    border: none !important;
    box-shadow: 0 4px 10px rgba(0, 0, 0, 0.2);
    transition: 0.3s ease-in-out !important;
  }
  .custom-button:hover {
    background: #0056b3 !important;
    transform: scale(1.05);
  }
  .header {
    color: white;
    text-align: center;
    font-size: 2.5em;
    font-weight: bold;
    padding: 20px 0;
  }
  .output-area {
    border-radius: 10px;
    padding: 15px;
    margin: 10px 0;
    background: #ffffff;
    box-shadow: 0 4px 8px rgba(0,0,0,0.1);
    text-align: center;
  }
</style>
"""))

title = widgets.HTML("<h1 class='header'>Steganography Tool</h1>")

upload_cover = widgets.FileUpload(description="Upload Cover Image", multiple=False)
upload_secret = widgets.FileUpload(description="Upload Secret Image", multiple=False)
encode_btn = widgets.Button(description="Encode Image", layout=widgets.Layout(width='300px', height='50px'))
view_encoded_btn = widgets.Button(description="View Encoded Image", layout=widgets.Layout(width='300px', height='50px'))
view_hidden_btn = widgets.Button(description="Reveal Secret Image", layout=widgets.Layout(width='300px', height='50px'))

for btn in [encode_btn, view_encoded_btn, view_hidden_btn]:
    btn.add_class('custom-button')

output = widgets.Output()
image_output = widgets.Output()

btn_layout = widgets.VBox([encode_btn, view_encoded_btn, view_hidden_btn], layout=widgets.Layout(align_items='flex-start'))
interface = widgets.VBox([
    title,
    upload_cover,
    upload_secret,
    btn_layout,
    image_output
], layout=widgets.Layout(align_items='flex-start', padding='20px'))

def encode_images(b):
    global encoded_image, decoded_image
    with output:
        clear_output()
        try:
            if not upload_cover.value or not upload_secret.value:
                raise ValueError("Please upload both cover and secret images!")

            cover_data = next(iter(upload_cover.value.values()))['content']
            secret_data = next(iter(upload_secret.value.values()))['content']

            cover = Image.open(io.BytesIO(cover_data)).convert("RGB")
            secret = Image.open(io.BytesIO(secret_data)).convert("RGB")

            secret = secret.resize(cover.size)

            cover_arr = np.array(cover)
            secret_arr = np.array(secret)

            cover_arr &= 0b11110000
            secret_bits = (secret_arr >> 4).astype(np.uint8)
            encoded_arr = cover_arr | secret_bits

            encoded_image = Image.fromarray(encoded_arr)
            decoded_image = decode_image(encoded_image)

            display(HTML("<h3 class='output-area'>Encoding Successful!</h3>"))
            add_download_buttons()
        except Exception as e:
            display(HTML(f"<h3 class='output-area' style='color:red;'>Error: {str(e)}</h3>"))

def decode_image(encoded_img):
    encoded_arr = np.array(encoded_img)
    secret_bits = (encoded_arr & 0b00001111).astype(np.uint8)
    return Image.fromarray(secret_bits << 4)

def show_encoded(b):
    with image_output:
        clear_output()
        if encoded_image:
            plt.figure(figsize=(10, 8))
            plt.imshow(encoded_image)
            plt.title("Encoded Image", fontsize=16)
            plt.axis('off')
            plt.show()
        else:
            display(HTML("<p class='output-area' style='color:red;'>Encode images first!</p>"))

def show_hidden(b):
    with image_output:
        clear_output()
        if decoded_image:
            plt.figure(figsize=(10, 8))
            plt.imshow(decoded_image)
            plt.title("Revealed Secret Image", fontsize=16)
            plt.axis('off')
            plt.show()
        else:
            display(HTML("<p class='output-area' style='color:red;'>Encode images first!</p>"))

def add_download_buttons():
    with image_output:
        clear_output()
        encoded_download_btn = widgets.Button(description="Download Encoded Image", layout=widgets.Layout(width='300px', height='50px'))
        secret_download_btn = widgets.Button(description="Download Secret Image", layout=widgets.Layout(width='300px', height='50px'))

        def on_encoded_download_click(b):
            encoded_image.save("encoded_image.png")
            files.download("encoded_image.png")

        def on_secret_download_click(b):
            decoded_image.save("decoded_secret.png")
            files.download("decoded_secret.png")

        encoded_download_btn.on_click(on_encoded_download_click)
        secret_download_btn.on_click(on_secret_download_click)
        display(widgets.HBox([encoded_download_btn, secret_download_btn], layout=widgets.Layout(justify_content='center')))

encode_btn.on_click(encode_images)
view_encoded_btn.on_click(show_encoded)
view_hidden_btn.on_click(show_hidden)

display(interface, output)


Output()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

image Staganography

Name: Shamir

Reg: 225150

Submitted To: Sir Ghulam Ali

Subject: DIP


# **Step-by-Step Working:**
# Uploading Images
The user uploads two images:
 Cover Image – The main image that will hide the secret image.
 Secret Image – The image that will be embedded inside the cover image.

# Image Processing & Resizing
The secret image is resized to match the dimensions of the cover image.

Both images are converted into NumPy arrays for pixel manipulation.

# Encoding the Secret Image (LSB Embedding)
The Least Significant Bits (LSBs) of the cover image are replaced with the Most Significant Bits (MSBs) of the secret image.

This subtle change keeps the cover image visually unchanged while hiding the secret image within.

# Example:

If a pixel in the cover image has a binary value of 11001100

And a pixel in the secret image has a binary value of 10100000

The final stego pixel will be 11001110 (embedding part of the secret image).

# Generating the Stego Image
The modified cover image (now containing the secret image) is saved as the Stego Image.

The user can download and share this encoded image.

# Extracting the Hidden Image (Decoding Process)
To retrieve the hidden image, the LSBs of the stego image are extracted and shifted back to their original position.

The extracted pixels reconstruct the original secret image with minor quality loss.

# Viewing & Downloading Results
The user can view the encoded stego image and decode the secret image using our tool.

# **Linked in Link:**
https://www.linkedin.com/posts/muhammad-shamir-10a130358_as-part-of-our-dip-course-with-ghulam-ali-activity-7310383049735655424-R8cW?utm_source=share&utm_medium=member_desktop&rcm=ACoAAFkB_5oBmKI8sFVNJi2pM3MHzinjrQn74Yc